# Inference with Jina Reader-LM Model Package

This notebook shows you how to deploy the [Jina Reader-LM](https://jina.ai/news/reader-lm-small-language-models-for-cleaning-and-converting-html-to-markdown) models using [ReaderLM v2 on Amazon SageMaker]([reader-lm-v2](https://aws.amazon.com/marketplace/pp/prodview-jwfct4j4rvxk2?sr=0-21&ref_=beagle&applicationId=AWSMPContessa)) and perform inference with it:


## Prerequisites:

1. This notebook should be rendered correctly in the Jupyter interface and can be executed either within an Amazon SageMaker Notebook Instance or Amazon SageMaker Studio.
2. Ensure that the IAM role being used has **AmazonSageMakerFullAccess**.
3. To successfully deploy this ML model, ensure that:
    1. Either your IAM role has the following three permissions and you have authority to make AWS Marketplace subscriptions in the AWS account:
        - **aws-marketplace:ViewSubscriptions**
        - **aws-marketplace:Unsubscribe**
        - **aws-marketplace:Subscribe**
    2. Or, your AWS account already has a subscription to this model.

# Contents:
1. [Model package setup](#model-package-setup)
2. [Inference](#-inference)
3. [Clean-up](#clean-up)
    1. [Delete the model](#delete-the-model)
    2. [Unsubscribe to the listing (optional)](#unsubscribe-to-the-listing-optional)

# Model package setup

Please first subscribe to the model package(s) from AWS Marketplace [here](https://aws.amazon.com/marketplace/pp/prodview-5iljbegvoi66w).

And then let's install `jina-sagemaker` package and get the model package ARN(s) using code below.

In [ ]:
!pip install --upgrade jina-sagemaker

import boto3

region = boto3.Session().region_name

# Specify the role as required by SageMaker
role = ""

# Model package name
model_package_name = "reader-lm-v2-be194f1fc4803c31ba8b28ffca3665b8"

# Mapping for product ARN
def get_arn_for_model(region_name, model_name):
    model_package_map = {
        "us-east-1": f"arn:aws:sagemaker:us-east-1:253352124568:model-package/{model_name}",
        "us-east-2": f"arn:aws:sagemaker:us-east-2:057799348421:model-package/{model_name}",
        "us-west-1": f"arn:aws:sagemaker:us-west-1:382657785993:model-package/{model_name}",
        "us-west-2": f"arn:aws:sagemaker:us-west-2:594846645681:model-package/{model_name}",
        "ca-central-1": f"arn:aws:sagemaker:ca-central-1:470592106596:model-package/{model_name}",
        "eu-central-1": f"arn:aws:sagemaker:eu-central-1:446921602837:model-package/{model_name}",
        "eu-west-1": f"arn:aws:sagemaker:eu-west-1:985815980388:model-package/{model_name}",
        "eu-west-2": f"arn:aws:sagemaker:eu-west-2:856760150666:model-package/{model_name}",
        "eu-west-3": f"arn:aws:sagemaker:eu-west-3:843114510376:model-package/{model_name}",
        "eu-north-1": f"arn:aws:sagemaker:eu-north-1:136758871317:model-package/{model_name}",
        "ap-southeast-1": f"arn:aws:sagemaker:ap-southeast-1:192199979996:model-package/{model_name}",
        "ap-southeast-2": f"arn:aws:sagemaker:ap-southeast-2:666831318237:model-package/{model_name}",
        "ap-northeast-2": f"arn:aws:sagemaker:ap-northeast-2:745090734665:model-package/{model_name}",
        "ap-northeast-1": f"arn:aws:sagemaker:ap-northeast-1:977537786026:model-package/{model_name}",
        "ap-south-1": f"arn:aws:sagemaker:ap-south-1:077584701553:model-package/{model_name}",
        "sa-east-1": f"arn:aws:sagemaker:sa-east-1:270155090741:model-package/{model_name}",
    }

    return model_package_map[region_name]

model_package_arn = get_arn_for_model(region, model_package_name)

---

# Inference

To learn about real-time inference capabilities in Amazon SageMaker, please refer to the [Documentation](https://docs.aws.amazon.com/sagemaker/latest/dg/realtime-endpoints.html).

Let's create an endpoint that uses Jina Reader-LM for inferencing.

In [ ]:
from jina_sagemaker import Client, InputType

client = Client(region_name=region)
endpoint_name = "my-endpoint"

We can create a new endpoint using the `create_endpoint` method and passing the required parameters like `instance_type`, `n_instances` etc.

In [ ]:
client.create_endpoint(
    arn=model_package_arn, 
    role=role, 
    endpoint_name=endpoint_name, 
    instance_type="ml.g4dn.xlarge", 
    n_instances=1,
)

Or, we can connect to an existing endpoint using the `connect_to_endpoint` method by passing the endpoint name.

In [ ]:
client.connect_to_endpoint(endpoint_name=endpoint_name, arn=model_package_arn)

Now let's try out the inference.

### Usage with `jina-sagemaker` sdk

First, construct the prompt using create_prompt, where you can specify the desired return format.

In [ ]:
def create_prompt(text: str, return_type: str, instruction: str = None, schema: str = None):
    """
    Creates a prompt based on the specified return type (either 'json' or 'markdown').
    
    Parameters:
    - text (str): The HTML content that needs to be converted into the desired format.
    - return_type (str): The desired return format. It must be either "json" or "markdown".
    - instruction (str): The instruction to be included in the prompt. If not provided, a default instruction is used.
    - schema (str): The JSON schema for structuring the output (used only for 'json' return_type). If empty, no schema is included.

    """
    
    if return_type not in ["json", "markdown"]:
        raise ValueError("Invalid return_type. Must be 'json' or 'markdown'.")
    
    if return_type == "json":
        if not instruction:
            instruction = "Extract the main content from the given HTML and convert it in a structured JSON format."

        if schema:
            prompt = f"{instruction}\n```html\n{text}\n```\nThe JSON schema is as follows:```json{schema}```" 
        else:
            prompt = f"{instruction}\n```html\n{text}\n```"
    
    elif return_type == "markdown":
        if not instruction:
            instruction = "Extract the main content from the given HTML and convert it to Markdown format."
        prompt = f"{instruction}\n```html\n{text}\n```"
    
    return prompt

html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Phone Book</title>
</head>
<body>
    <h1>Phone Book</h1>
    <div class="contact">
        <h2>John Doe</h2>
        <p>Email: <a href="mailto:john.doe@example.com">john.doe@example.com</a></p>
    </div>
    <div class="contact">
        <h2>Jane Smith</h2>
        <p>Email: <a href="mailto:jane.smith@example.com">jane.smith@example.com</a></p>
    </div>
</body>
</html>
"""

prompt = create_prompt(return_type="markdown", text=html)
print(prompt)

In [ ]:
result = client.read(prompt=prompt)

### Usage with aws-sdk

Create a input file `input.json` with the following content.

```json
{
    "model": "ReaderLM-v2",
    "prompt": "Extract the main content from the given HTML and convert it to Markdown format.\n```html\n<!DOCTYPE html><html lang=\"en\"><head><meta charset=\"UTF-8\"><meta name=\"viewport\" content=\"width=device-width, initial-scale=1.0\"><title>Phone Book</title></head><body><h1>Phone Book</h1><div class=\"contact\"><h2>John Doe</h2><p>Email: <a href=\"mailto:john.doe@example.com\">john.doe@example.com</a></p></div><div class=\"contact\"><h2>Jane Smith</h2><p>Email: <a href=\"mailto:jane.smith@example.com\">jane.smith@example.com</a></p></div></body></html>\n```",
    "stream": false
}
```

Run the AWS `invoke-endpoint` CLI.

In [ ]:
aws sagemaker-runtime invoke-endpoint \
--endpoint-name <endpoint-name> \
--content-type 'application/json' \
--body fileb://input.json \
output.json

# Clean-up

## Delete the model

In [ ]:
client.delete_endpoint()
client.close()

## Unsubscribe to the listing (optional)

If you would like to unsubscribe to the model package, follow these steps. Before you cancel the subscription, ensure that you do not have any [deployable model](https://console.aws.amazon.com/sagemaker/home#/models) created from the model package or using the algorithm. Note - You can find this information by looking at the container name associated with the model. 

**Steps to unsubscribe to product from AWS Marketplace**:
1. Navigate to __Machine Learning__ tab on [__Your Software subscriptions page__](https://aws.amazon.com/marketplace/ai/library?productType=ml&ref_=mlmp_gitdemo_indust)
2. Locate the listing that you want to cancel the subscription for, and then choose __Cancel Subscription__  to cancel the subscription.
